<a href="https://colab.research.google.com/github/andreschen33/Yumera/blob/main/ASRNEMO_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONNECTING TO DRIVE AND BUILD THE MODEL

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nb_path ='/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)
"""place the nb_path in the middle section of install pip like examples : 
    pip install --target==$nb_path blablabla"""
    

https://stackoverflow.com/questions/55253498/how-do-i-install-a-library-permanently-in-colab

# INSTALLING DEPENDENCIES

In [ ]:
## Install dependencies
!pip install wget
!apt-get install libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install torchaudio
## Install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]


## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/config.yaml


In [ ]:
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

In [ ]:
pip install imgaug==0.2.5

In [ ]:
pip install Sphinx==2.0.0

In [ ]:
pip install folium==0.2.1

In [ ]:
pip install Sphinx==2.0.0

In [ ]:
pip install urllib3==1.25.4

In [ ]:
## Install dependencies
!pip install wget
!apt-get install libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2
!pip install torchaudio
## Install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/config.yaml

In [ ]:
!pip install ffmpeg-python

# IMPORTING NECESSARY MODULES

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts

[NeMo W 2021-01-14 03:55:36 experimental:28] Module <class 'nemo.collections.asr.losses.ctc.CTCLoss'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


[NeMo W 2021-01-14 03:55:40 experimental:28] Module nemo.collections.asr.data.audio_to_text.AudioToCharDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-01-14 03:55:40 experimental:28] Module nemo.collections.asr.data.audio_to_text.AudioToBPEDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-01-14 03:55:40 experimental:28] Module nemo.collections.asr.data.audio_to_text.AudioLabelDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-01-14 03:55:40 experimental:28] Module nemo.collections.asr.data.audio_to_text.TarredAudioToTextDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-01-14 03:55:40 experimental:28] Module nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset is experimental, not ready for production and is not fully supporte

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
# We'll use this to listen to audio
import IPython

# VOICE CODE MODELS


https://colab.research.google.com/drive/1Z6VIRZ_sX314hyev3Gm5gBqvm1wQVo-a

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr



# BUILDING THE SPEECH RECOGNITION MODELS


IMPORTING SPECTROGRAM QUARTZNET AND OTHERS

In [ ]:
# Automatic Speech Recognition Models
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En",strict=False)
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='Punctuation_Capitalization_with_DistilBERT', strict = False)
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="Tacotron2-22050Hz", strict = False)
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz", strict =False)


[NeMo I 2021-01-14 03:55:42 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b4/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo
[NeMo I 2021-01-14 03:55:43 common:423] Instantiating model from pre-trained checkpoint
[NeMo I 2021-01-14 03:55:44 features:236] PADDING: 16
[NeMo I 2021-01-14 03:55:44 features:252] STFT using torch
[NeMo I 2021-01-14 03:55:44 modelPT:402] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-01-14 03:55:44 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemonlpmodels/versions/1.0.0a5/files/Punctuation_Capitalization_with_DistilBERT.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b4/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/P

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


[NeMo W 2021-01-14 03:55:53 modelPT:146] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-01-14 03:55:53 modelPT:153] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    ds_item: null
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-01-14 03:55:53 modelPT:1175] World size can only be set by PyTorch Lightning Trainer.



[NeMo I 2021-01-14 03:56:07 modelPT:402] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo.
[NeMo I 2021-01-14 03:56:07 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemottsmodels/versions/1.0.0a5/files/Tacotron2-22050Hz.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b4/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo
[NeMo I 2021-01-14 03:56:10 common:423] Instantiating model from pre-trained checkpoint
[NeMo I 2021-01-14 03:56:11 features:236] PADDING: 16
[NeMo I 2021-01-14 03:56:11 features:245] STFT using conv
[NeMo I 2021-01-14 03:56:13 modelPT:402] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo.
[NeMo I 2021-01-14 03:56:13 cloud:66] Downloading

AUDIO RECORDING 

In [ ]:
audio, sr = get_audio()
import scipy 
scipy.io.wavfile.write('a.wav', sr, audio)

In [ ]:
filesx = "/content/a.wav"
files = [filesx]
IPython.display.Audio('a.wav')

BUILD SPEECH RECOGNITION

In [ ]:
# SPEECH RECOGNITION TRANSCRIPTING
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

BUILD PUNCTUATION

In [ ]:
# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]

BUILD SPECTOGRAM AND VOCODER

In [ ]:
# A helper function which combines Tacotron2 and WaveGlow to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').numpy()

OUTPUT

In [ ]:
# Without punctuation
print('-------> Without Punctuation')
print(f"Audio in {fname} was recognized as: {raw_text}")
IPython.display.Audio(text_to_audio(raw_text), rate=27000)

In [ ]:
# with Punctuation
print('-------> With Punctuation')
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')
IPython.display.Audio(text_to_audio(text), rate = 27000)

# BUILD SPEECH COMMAND MODELS

https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/04_Online_Offline_Speech_Commands_Demo.ipynb

In [ ]:
import librosa
import numpy as np
import os, time
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
from omegaconf import OmegaConf
import copy

In [ ]:
mbn_model =nemo_asr.models.EncDecClassificationModel.from_pretrained(model_name='MatchboxNet-3x2x64-v1')

[NeMo I 2021-01-13 12:45:08 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/MatchboxNet-3x2x64-v1.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b4/MatchboxNet-3x2x64-v1/8a0833bb977e90f94738e2c6c3da61f3/MatchboxNet-3x2x64-v1.nemo
[NeMo I 2021-01-13 12:45:10 common:423] Instantiating model from pre-trained checkpoint


[NeMo W 2021-01-13 12:45:10 modelPT:146] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    labels:
    - bed
    - bird
    - cat
    - dog
    - down
    - eight
    - five
    - four
    - go
    - happy
    - house
    - left
    - marvin
    - nine
    - 'no'
    - 'off'
    - 'on'
    - one
    - right
    - seven
    - sheila
    - six
    - stop
    - three
    - tree
    - two
    - up
    - wow
    - 'yes'
    - zero
    batch_size: 128
    shuffle: true
    augmentor:
      shift:
        prob: 1.0
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 1.0
        min_level: -90
        max_level: -46
    
[NeMo W 2021-01-13 12:45:10 modelPT:153] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the vali

[NeMo I 2021-01-13 12:45:10 modelPT:402] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/MatchboxNet-3x2x64-v1/8a0833bb977e90f94738e2c6c3da61f3/MatchboxNet-3x2x64-v1.nemo.


In [ ]:
vad_model = nemo_asr.models.EncDecClassificationModel.from_pretrained('MatchboxNet-VAD-3x2')

[NeMo I 2021-01-13 12:45:10 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/MatchboxNet_VAD_3x2.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b4/MatchboxNet_VAD_3x2/1375f3813383105a24acc75428ec51c4/MatchboxNet_VAD_3x2.nemo
[NeMo I 2021-01-13 12:45:12 common:423] Instantiating model from pre-trained checkpoint


[NeMo W 2021-01-13 12:45:12 modelPT:146] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/fjia/code/manifest64/balanced_background_training_manifest.json,/home/fjia/code/manifest64/balanced_speech_training_manifest.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 128
    num_workers: 20
    shuffle: true
    augmentor:
      shift:
        prob: 0.8
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 0.8
        min_level: -90
        max_level: -46
    
[NeMo W 2021-01-13 12:45:12 modelPT:153] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/fjia/code/manifest64/balanced_background_validation_manifest.json,/home/

[NeMo I 2021-01-13 12:45:13 modelPT:402] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b4/MatchboxNet_VAD_3x2/1375f3813383105a24acc75428ec51c4/MatchboxNet_VAD_3x2.nemo.


In [ ]:
# Preserve a copy of the full config
vad_cfg = copy.deepcopy(vad_model._cfg)
mbn_cfg = copy.deepcopy(mbn_model._cfg)
print(OmegaConf.to_yaml(mbn_cfg))

# Set model to inference mode
mbn_model.eval();
vad_model.eval();

timesteps: 128
repeat: 2
dropout: 0.0
kernel_size_factor: 1.0
labels:
- bed
- bird
- cat
- dog
- down
- eight
- five
- four
- go
- happy
- house
- left
- marvin
- nine
- 'no'
- 'off'
- 'on'
- one
- right
- seven
- sheila
- six
- stop
- three
- tree
- two
- up
- wow
- 'yes'
- zero
train_ds:
  manifest_filepath: null
  sample_rate: 16000
  labels:
  - bed
  - bird
  - cat
  - dog
  - down
  - eight
  - five
  - four
  - go
  - happy
  - house
  - left
  - marvin
  - nine
  - 'no'
  - 'off'
  - 'on'
  - one
  - right
  - seven
  - sheila
  - six
  - stop
  - three
  - tree
  - two
  - up
  - wow
  - 'yes'
  - zero
  batch_size: 128
  shuffle: true
  augmentor:
    shift:
      prob: 1.0
      min_shift_ms: -5.0
      max_shift_ms: 5.0
    white_noise:
      prob: 1.0
      min_level: -90
      max_level: -46
validation_ds:
  manifest_filepath: null
  sample_rate: 16000
  labels:
  - bed
  - bird
  - cat
  - dog
  - down
  - eight
  - five
  - four
  - go
  - happy
  - house
  - left
  - m

SETTING UP DATA FOR STREAMING INFERENCE

In [ ]:
from nemo.core.classes import IterableDataset
from nemo.core.neural_types import NeuralType, AudioSignal, LengthsType
import torch
from torch.utils.data import DataLoader

In [ ]:
# simple data layer to pass audio signal
class AudioDataLayer(IterableDataset):
    @property
    def output_types(self):
        return {
            'audio_signal': NeuralType(('B', 'T'), AudioSignal(freq=self._sample_rate)),
            'a_sig_length': NeuralType(tuple('B'), LengthsType()),
        }

    def __init__(self, sample_rate):
        super().__init__()
        self._sample_rate = sample_rate
        self.output = True
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if not self.output:
            raise StopIteration
        self.output = False
        return torch.as_tensor(self.signal, dtype=torch.float32), \
               torch.as_tensor(self.signal_shape, dtype=torch.int64)
        
    def set_signal(self, signal):
        self.signal = signal.astype(np.float32)/32768.
        self.signal_shape = self.signal.size
        self.output = True

    def __len__(self):
        return 1

In [ ]:
data_layer = AudioDataLayer(sample_rate=mbn_cfg.train_ds.sample_rate)
data_loader = DataLoader(data_layer, batch_size=1, collate_fn=data_layer.collate_fn)

INFERENCE METHOD FOR AUDIO SIGNAL

In [ ]:
def infer_signal(model, signal):
    data_layer.set_signal(signal)
    batch = next(iter(data_loader))
    audio_signal, audio_signal_len = batch
    audio_signal, audio_signal_len = audio_signal.to(model.device), audio_signal_len.to(model.device)
    logits = model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
    return logits

In [ ]:
# class for streaming frame-based ASR
# 1) use reset() method to reset FrameASR's state
# 2) call transcribe(frame) to do ASR on
#    contiguous signal's frames
class FrameASR:
    
    def __init__(self, model_definition,
                 frame_len=2, frame_overlap=2.5, 
                 offset=0):
        '''
        Args:
          frame_len (seconds): Frame's duration
          frame_overlap (seconds): Duration of overlaps before and after current frame.
          offset: Number of symbols to drop for smooth streaming.
        '''
        self.task = model_definition['task']
        self.vocab = list(model_definition['labels'])
        
        self.sr = model_definition['sample_rate']
        self.frame_len = frame_len
        self.n_frame_len = int(frame_len * self.sr)
        self.frame_overlap = frame_overlap
        self.n_frame_overlap = int(frame_overlap * self.sr)
        timestep_duration = model_definition['AudioToMFCCPreprocessor']['window_stride']
        for block in model_definition['JasperEncoder']['jasper']:
            timestep_duration *= block['stride'][0] ** block['repeat']
        self.buffer = np.zeros(shape=2*self.n_frame_overlap + self.n_frame_len,
                               dtype=np.float32)
        self.offset = offset
        self.reset()
        
    @torch.no_grad()
    def _decode(self, frame, offset=0):
        assert len(frame)==self.n_frame_len
        self.buffer[:-self.n_frame_len] = self.buffer[self.n_frame_len:]
        self.buffer[-self.n_frame_len:] = frame

        if self.task == 'mbn':
            logits = infer_signal(mbn_model, self.buffer).to('cpu').numpy()[0]
            decoded = self._mbn_greedy_decoder(logits, self.vocab)
            
        elif self.task == 'vad':
            logits = infer_signal(vad_model, self.buffer).to('cpu').numpy()[0]
            decoded = self._vad_greedy_decoder(logits, self.vocab)
           
        else:
            raise("Task should either be of mbn or vad!")
            
        return decoded[:len(decoded)-offset]
    
    def transcribe(self, frame=None,merge=False):
        if frame is None:
            frame = np.zeros(shape=self.n_frame_len, dtype=np.float32)
        if len(frame) < self.n_frame_len:
            frame = np.pad(frame, [0, self.n_frame_len - len(frame)], 'constant')
        unmerged = self._decode(frame, self.offset)
        return unmerged
        
    
    def reset(self):
        '''
        Reset frame_history and decoder's state
        '''
        self.buffer=np.zeros(shape=self.buffer.shape, dtype=np.float32)
        self.mbn_s = []
        self.vad_s = []
        
    @staticmethod
    def _mbn_greedy_decoder(logits, vocab):
        mbn_s = []
        if logits.shape[0]:
            class_idx = np.argmax(logits)
            class_label = vocab[class_idx]
            mbn_s.append(class_label)         
        return mbn_s
    
    
    @staticmethod
    def _vad_greedy_decoder(logits, vocab):
        vad_s = []
        if logits.shape[0]:
            probs = torch.softmax(torch.as_tensor(logits), dim=-1)
            probas, preds = torch.max(probs, dim=-1)
            vad_s = [preds.item(), str(vocab[preds]), probs[0].item(), probs[1].item(), str(logits)]
        return vad_s

STREAMING INFERENCE

In [ ]:
def listToString(s):  
    str1 = " " 
    return (str1.join(s)) 

In [ ]:
import wave

def offline_inference(wave_file, STEP = 0.25, WINDOW_SIZE = 0.31):
    """
    Arg:
        wav_file: wave file to be performed inference on.
        STEP: infer every STEP seconds 
        WINDOW_SIZE : lenght of audio to be sent to NN.
    """
    
    FRAME_LEN = STEP 
    CHANNELS = 1 # number of audio channels (expect mono signal)
    RATE = SAMPLE_RATE # sample rate, 16000 Hz
   
    CHUNK_SIZE = int(FRAME_LEN * SAMPLE_RATE)
    
    mbn = FrameASR(model_definition = {
                       'task': 'mbn',
                       'sample_rate': SAMPLE_RATE,
                       'AudioToMFCCPreprocessor': mbn_cfg.preprocessor,
                       'JasperEncoder': mbn_cfg.encoder,
                       'labels': mbn_cfg.labels
                   },
                   frame_len=FRAME_LEN, frame_overlap = (WINDOW_SIZE - FRAME_LEN)/2,
                   offset=0)

    wf = wave.open(wave_file, 'rb')
    data = wf.readframes(CHUNK_SIZE)

    while len(data) > 0:

        data = wf.readframes(CHUNK_SIZE)
        signal = np.frombuffer(data, dtype=np.int16)
        mbn_result = mbn.transcribe(signal)
        mbn_result = listToString(mbn_result)

        if len(mbn_result):
          print(mbn_result)


    mbn.reset()
    return mbn_result

In [ ]:
import wave
import contextlib
fname = filesx
with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
    print(duration)

4.98


In [ ]:
STEP =0.16 #training size step (more step equal more accurate standard 0.25)
WINDOW_SIZE = duration # input segment length for NN we used for training 1.28
# sample rate, Hz
SAMPLE_RATE = 16000
demo_wave ='a.wav'

# Ground-truth is Yes No
result = offline_inference(demo_wave, STEP, WINDOW_SIZE)
print(result)

[NeMo W 2021-01-13 12:45:13 nemo_logging:349] /usr/local/lib/python3.6/dist-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
      normalized, onesided, return_complex)
    


up
up
up
sheila
eight
up
up
up
up
up
up
up
up
six
up
up
up
up
eight
nine
up
eight
six
off
zero
up
off
off
nine
nine
off
off
off
left
eight
two
off
up
up
marvin
eight
two
off
off
off
off
off
on
two
off
no
off
on
three
on
eight
off
off
left
off
no
zero
off
no
two
eight
no
left
eight
one
on
left
eight
no
one
off
up
eight
eight
left
left
off
eight
off
on
one
on
off
eight
eight
up
on
off
eight
eight


In [ ]:

IPython.display.Audio(text_to_audio(result), rate = 27000)